In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [122]:
# Store csv data into DataFrame
csv_file = "./data_sources/combined_data_clean.csv"
combined_data_df = pd.read_csv(csv_file)
combined_data_df.head()

,Name,county_state,date1,Serial,date2,Pre_rally_beg,pre_rally_end,post_rally_beg,post_rally_end,average_growth_new_cases,pre_rally_rate,post_rally_rate,post_rally_growth_new_cases,post_rally_change_in_growth,pre_rally_rate.1,post_rally_rate.1,post_rally_growth_deaths
0,Joe Biden,Milwaukee_Wisconsin,20200903,Milwaukee_Wisconsin_20200903,9/3/2020,8/20/2020,9/3/2020,9/3/2020,9/17/2020,8.8%,330503,399423,20.85%,135.99%,6777,7619,12.42%
1,Donald Trump,Westmoreland_Pennsylvania,20200903,Westmoreland_Pennsylvania_20200903,9/3/2020,8/20/2020,9/3/2020,9/3/2020,9/17/2020,28.6%,24721,28982,17.24%,-39.69%,686,736,7.29%
2,Joe Biden,Dauphin_Pennsylvania,20200907,Dauphin_Pennsylvania_20200907,9/7/2020,8/24/2020,9/7/2020,9/7/2020,9/21/2020,-2.5%,46296,53798,16.20%,-746.35%,2284,2543,11.34%
3,Donald Trump,Forsyth_North Carolina,20200908,Forsyth_North Carolina_20200908,9/8/2020,8/25/2020,9/8/2020,9/8/2020,9/22/2020,-3.4%,86603,99921,15.38%,-555.72%,1068,1342,25.66%
4,Donald Trump,Palm Beach_Florida,20200908,Palm Beach_Florida_20200908,9/8/2020,8/25/2020,9/8/2020,9/8/2020,9/22/2020,-11.9%,589620,665859,12.93%,-208.91%,15758,18412,16.84%


In [123]:
# Clean rally and candidate DataFrames
combined_data_df.rename(columns={'Name': 'name', 'date2': 'date', 'Pre_rally_beg': 'pre_rally_date', 
                                'post_rally_end': 'post_rally_date', 'average_growth_new_cases': 'pre_rally_case_growth_rate',
                                'pre_rally_rate': 'pre_rally_num_cases', 'post_rally_rate': 'post_rally_num_cases',
                                'post_rally_growth_new_cases': 'post_rally_case_growth_rate',
                                'post_rally_change_in_growth': 'case_growth_rate_change',
                                'pre_rally_rate.1': 'pre_rally_num_deaths',
                                'post_rally_rate.1': 'post_rally_num_deaths',
                                'post_rally_growth_deaths': 'death_growth_rate_change'}, inplace=True)
# Convert percent to float
combined_data_df['pre_rally_case_growth_rate'] = combined_data_df['pre_rally_case_growth_rate'].str.rstrip('%').astype('float') / 100.0
combined_data_df['post_rally_case_growth_rate'] = combined_data_df['post_rally_case_growth_rate'].str.rstrip('%').astype('float') / 100.0
combined_data_df['case_growth_rate_change'] = combined_data_df['case_growth_rate_change'].str.rstrip('%').astype('float') / 100.0
combined_data_df['death_growth_rate_change'] = combined_data_df['death_growth_rate_change'].str.rstrip('%').astype('float') / 100.0

# print(combined_data_df)


# Normalize candidates
rally_df = combined_data_df[["county_state", "date"]].copy()
# rally_df.head()

rally_candidate_df = combined_data_df[["name"]].copy()
print(rally_candidate_df)

candidate_list = []
rally_candidate = []


# Candidates are NOT hard-coded, we allow for any variable number of candidates.

# Loop through candidate_df, if current candidate is not in candidate list, add to list.
# Add and populate new candidate_ID column in rally_df.
for row in rally_candidate_df.itertuples():
    if not row[1] in candidate_list: # If candidate is not already in candidate_list,
        candidate_list.append(row[1]) # Then append candidate to candidate_list.
    rally_candidate.append(candidate_list.index(row[1]) + 1) # Track candidate_id foreign key. +1 because pgAdmin candidate table index starts at 1.

rally_df = rally_df.assign(candidate_id = rally_candidate)

print(rally_df)
# print(candidate_list)

candidate_df = pd.DataFrame(candidate_list,columns=['name'])

# print(rally_candidate)
print(candidate_df)

             name
0       Joe Biden
1    Donald Trump
2       Joe Biden
3    Donald Trump
4    Donald Trump
..            ...
102  Donald Trump
103  Donald Trump
104  Donald Trump
105     Joe Biden
106     Joe Biden

[107 rows x 1 columns]
                  county_state       date  candidate_id
0          Milwaukee_Wisconsin   9/3/2020             1
1    Westmoreland_Pennsylvania   9/3/2020             2
2         Dauphin_Pennsylvania   9/7/2020             1
3       Forsyth_North Carolina   9/8/2020             2
4           Palm Beach_Florida   9/8/2020             2
..                         ...        ...           ...
102    Grand Traverse_Michigan  11/2/2020             2
103              Kent_Michigan  11/2/2020             2
104       Luzerne_Pennsylvania  11/2/2020             2
105        New Castle_Delaware  11/3/2020             1
106  Philadelphia_Pennsylvania  11/3/2020             1

[107 rows x 3 columns]
           name
0     Joe Biden
1  Donald Trump


In [10]:
# Connect to local database
rds_connection_string = "postgres:password!@localhost:5432/covid_campaign" # Must have existing 'covid_campaign' database in pgAdmin
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['rally', 'candidate', 'pre_rally_covid_stats', 'post_rally_covid_stats']

In [105]:
# Create database tables
# Execute the following SQL code through the query editor in pgAdmin:

# DROP TABLE IF EXISTS post_rally_covid_stats CASCADE; -- Drop and import/create in reverse orders to satisfy constraints
# DROP TABLE IF EXISTS pre_rally_covid_stats CASCADE;
# DROP TABLE IF EXISTS candidate CASCADE;
# DROP TABLE IF EXISTS rally CASCADE;

# CREATE TABLE rally (
# 	rally_id SERIAL PRIMARY KEY,
# 	candidate_id INT,
# 	county_state TEXT,
# 	date DATE, -- Date of rally, end of pre window, start of post window
# 	pre_rally_covid_stats_id INT,
# 	post_rally_covid_stats_id INT,
# 	case_growth_rate_change DECIMAL,
# 	death_growth_rate_change DECIMAL
# );

# CREATE TABLE candidate (
# 	candidate_id SERIAL PRIMARY KEY,
# 	name TEXT
# );

# CREATE TABLE pre_rally_covid_stats (
# 	pre_rally_covid_stats_id SERIAL PRIMARY KEY,
# 	pre_rally_date DATE, -- Pre window start date
# 	pre_rally_num_cases INT,
# 	pre_rally_case_growth_rate DECIMAL,
# 	pre_rally_num_deaths INT
# );

# CREATE TABLE post_rally_covid_stats (
# 	post_rally_covid_stats_id SERIAL PRIMARY KEY,
# 	post_rally_date DATE, -- Post window end date
# 	post_rally_num_cases INT,
# 	post_rally_case_growth_rate DECIMAL,
# 	post_rally_num_deaths INT
# );

In [106]:
# Load cleaned rally and candidate DataFrames into database
rally_df.to_sql(name='rally', con=engine, if_exists='append', index=False)
candidate_df.to_sql(name='candidate', con=engine, if_exists='append', index=False)

In [118]:
# Clean pre_stats and post_stats DataFrames
pre_rally_covid_stats_df = combined_data_df[["pre_rally_date", "pre_rally_num_cases", 
                                             "pre_rally_case_growth_rate", "pre_rally_num_deaths"]].copy()
print(pre_rally_covid_stats_df.head())

post_rally_covid_stats_df = combined_data_df[["post_rally_date", "post_rally_num_cases", 
                                             "post_rally_case_growth_rate", "post_rally_num_deaths"]].copy()
print(post_rally_covid_stats_df.head())

  pre_rally_date  pre_rally_num_cases  pre_rally_case_growth_rate  \
0      8/20/2020               330503                       0.088   
1      8/20/2020                24721                       0.286   
2      8/24/2020                46296                      -0.025   
3      8/25/2020                86603                      -0.034   
4      8/25/2020               589620                      -0.119   

   pre_rally_num_deaths  
0                  6777  
1                   686  
2                  2284  
3                  1068  
4                 15758  
  post_rally_date  post_rally_num_cases  post_rally_case_growth_rate  \
0       9/17/2020                399423                       0.2085   
1       9/17/2020                 28982                       0.1724   
2       9/21/2020                 53798                       0.1620   
3       9/22/2020                 99921                       0.1538   
4       9/22/2020                665859                       0.1293 

In [120]:
# Load cleaned pre_stats and post_stats DataFrames into database
pre_rally_covid_stats_df.to_sql(name='pre_rally_covid_stats', con=engine, if_exists='append', index=False)
post_rally_covid_stats_df.to_sql(name='post_rally_covid_stats', con=engine, if_exists='append', index=False)